In [23]:
import json
import pandas as pd
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt

from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import TerminalFormatter

from google_play_scraper import Sort, reviews, app

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

app_packages = [
    'com.google.android.apps.meetings',
    'com.instagram.android',
    'com.microsoft.teams',
    'com.netflix.mediaclient',
    'com.zhiliaoapp.musically',
    'com.whatsapp',
    'us.zoom.videomeetings'
]

app_infos = []

for ap in tqdm(app_packages):
  info = app(ap, lang='en', country='us')
  del info['comments']
  app_infos.append(info)    

app_infos_df = pd.DataFrame(app_infos)
app_infos_df.to_csv('apps.csv', index=None, header=True)

app_reviews = []

for ap in tqdm(app_packages):
  for score in list(range(1, 6)):
    for sort_order in [Sort.NEWEST]:
      rvs, _ = reviews(
        ap,
        lang='en',
        country='us',
        sort=sort_order,
        count= 10000,
        filter_score_with=score
      )
      for r in rvs:
        r['sortOrder'] = 'most_relevant' if sort_order == Sort.MOST_RELEVANT else 'newest'
        r['appId'] = ap
      app_reviews.extend(rvs)

app_reviews_df = pd.DataFrame(app_reviews)
app_reviews_df.to_csv('reviews.csv', index=None, header=True)

100%|███████████████████████████████████████████████████████████████████████████████████| 7/7 [42:37<00:00, 365.32s/it]


In [9]:
import pandas as pd
data = pd.read_csv("reviews_google_meet_v2.csv")
data_text = data[['content']]
data_text['index'] = data_text.index
documents = data_text

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer
import numpy as np

def lemmatize_only(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(str(text).encode('ascii',errors='ignore').decode()):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(token)
    return result

processed_docs = documents['content'].map(preprocess)
bigram = gensim.models.Phrases(processed_docs, min_count=1, threshold=1)
bigram_mod = gensim.models.phrases.Phraser(bigram)
bigram_docs = list()
for processed_doc in processed_docs:
    bigram_docs.append(bigram_mod[processed_doc])
dictionary = gensim.corpora.Dictionary(bigram_docs)
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=5000)
bow_corpus = [dictionary.doc2bow(doc) for doc in bigram_docs]

from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

print('\nBag of Words Model:\n')
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
    
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

print('\nTF-IDF Model:\n')
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))


Bag of Words Model:

Topic: 0 
Words: 0.054*"like" + 0.053*"great" + 0.048*"excellent" + 0.021*"easy" + 0.020*"video" + 0.019*"improve" + 0.017*"meeting" + 0.017*"apps" + 0.015*"time" + 0.014*"experience"
Topic: 1 
Words: 0.049*"love" + 0.038*"meetings" + 0.023*"good" + 0.022*"classes" + 0.018*"meeting" + 0.016*"school" + 0.015*"video" + 0.013*"google_meet" + 0.011*"video_calling" + 0.010*"online"
Topic: 2 
Words: 0.085*"best" + 0.034*"meeting" + 0.032*"good_online" + 0.026*"good" + 0.025*"classes" + 0.023*"online_class" + 0.019*"zoom" + 0.016*"google_meet" + 0.016*"download" + 0.015*"class"
Topic: 3 
Words: 0.045*"better" + 0.027*"helpful" + 0.026*"audio" + 0.024*"online_classes" + 0.018*"video_quality" + 0.018*"meeting" + 0.017*"meet" + 0.016*"thanks" + 0.014*"best_meeting" + 0.014*"classes"
Topic: 4 
Words: 0.611*"good" + 0.016*"screen" + 0.015*"awesome" + 0.012*"meeting" + 0.009*"like" + 0.007*"video" + 0.007*"features" + 0.006*"data" + 0.005*"chat" + 0.005*"problem"
Topic: 5 
Wor